<a href="https://colab.research.google.com/github/Mabinogit/AI-Image-Classification/blob/main/RAG_fundementals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import openai
from dotenv import load_dotenv
import chromadb
from openai import OpenAI
from chromadb.utils import embedding_functions


In [ ]:

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_key,
    model_name="text-embedding-ada-002"
)

In [ ]:
# INITIALIZE THE CHROMA CLIEN WITH PERSISTENCE

chroam_client = chromadb.PersistentClient(path="./chroma_db")
collection_name = "rag_collection"
collection = chroam_client.get_or_create_collection(name=collection_name, embedding_function=openai_ef)


# Load Data

# Function to load data into list
I want to be able to not only load text files but also pdf files this includes files that contian words and pictures

In [86]:
# Function to load documents from directory

def load_documents(directory):
  print(f"Loading documents from {directory}")
  documents = []
  for filename in os.listdir(directory):
    if filename.endswith(".txt"):
      with open(os.path.join(directory, filename), "r",) as f:
        content = f.read()
        documents.append(content)
  return documents





# Funtion to split data into chunks
I want to be able to split all kind of data not only text into chunks

In [88]:
# the more overlap the more context will be kept because the content will overlap
def split_text(text, chunk_size=1000, chunk_overlap=20):
  print(f"Splitting text into chunks of {chunk_size} characters, with {chunk_overlap} characters overlap")
  chunks = []
  start = 0
  while start < len(text):
    end = start + chunk_size
    chunks.append(text[start:end])
    start = end - chunk_overlap
  print(f"Split text into {len(chunks)} chunks")
  return chunks

In [90]:
# load data from directory
directory = "./data"
documents = load_documents(directory)

Loading documents from ./data


FileNotFoundError: [Errno 2] No such file or directory: './data'

In [91]:
#split loaded documents into chunks


chunked_documents = []
for doc in documents:
  chunks = split_text(doc["text"])
  print("=== splitting docs into chunks===")
  for i, chunk in enumerate(chunks):
    chunked_documents.append({"id": f"{doc['id']})-{i}", "text": chunk})

print(f"split documents int {len(chunked_documents)} chuncks")

NameError: name 'documents' is not defined

# Function to transform chunks into embeddings

Be able to embedd all kinds of data to be stored into vector data base

Downloaded model to handel all embedding - all-MiniLM-L6-v2

In [77]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"
filesnames = [ "config.json","config_sentence_transformers.json","data_config.json",
              "model.safetensors","modules.json", "pytorch_model.bin","rust_model.ot",
               "sentence_bert_config.json","special_tokens_map.json","tf_model.h5",
               "tokenizer.json","tokenizer_config.json","train_script.py",
               "vocab.txt"
]

for filename in filesnames:
  download_path = hf_hub_download(repo_id=model_id,filename = filename, token = hugging_api)
  print(download_path)



Function to generate embedding using all-MiniLM-L6-v2

In [81]:
def getall_MiniLM_L6_v2_embeddings(text):
  from sentence_transformers import SentenceTransformer

  model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  embeddings = model.encode(text)
  print("=== GENERATING EMBEDDINGS ===")
  return embeddings

In [84]:
# Generate embedding for the document chuncks
for doc in chunks:
  print("=== GENERATING EMBEDDINGS ===")
  doc["embedding"] = getall_MiniLM_L6_v2_embeddings(doc["text"])
print("=== EMBEDDINGS GENERATED ===")

NameError: name 'chunks' is not defined

# Storing data into Vector Database

We are going to have a data base that has both the chunked and and its equvillent embedded data

In [92]:
# upsert documents with embedding into Chromadb

for doc in chuncked_documents:
  print("=== inserting chunks into db ;;; ===")
  collection.upsert( ids=[doc["id"]],
                    documents=[doc["text"]],
                    embeddings=[doc["embedding"]])
print("=== DOCUMENTS UPSERTED ===")

NameError: name 'chuncked_documents' is not defined

# Searching for data In Vector DataBase

In [94]:
# function to query our documents. look for document that closest to the user query

def query_documents(question, n_results = 2):
  # query_embedding = get all-Mini_embedding(question)
  results =collection.quesry(query_texts = question, n_results = n_results)

  #Extract the relevent chunks
  relevent_chunks = [doc for sublist in results["documents"] for doc in sublist]
  print("===Returninh relevent chunks ===")
  return relevent_chunks


# Generating relevent Response

Logging in to hugging face and accessing model

In [ ]:
import os
from huggingface_hub import hf_hub_download

from google.colab import userdata
hugging_api = userdata.get('Hugging_rag_api')

# login into hugging face client
from huggingface_hub import login
login(token=hugging_api)


Downloading model

In [97]:
model_id = "openai-community/gpt2"
filesnames = [ "config.json" ,"flax_model.msgpack" ,"merges.txt" ,
              "model.safetensors" ,"pytorch_model.bin" ,"rust_model.ot" ,"tf_model.h5",
              "tokenizer.json" ,"tokenizer_config.json" ,"vocab.json" ,
              "64-fp16.tflite" ,"64-fp16.tflite" ,"64.tflite"
]


# Download LLM Files
for filename in filesnames:
  download_path = hf_hub_download(repo_id=model_id,filename=filename, token = hugging_api)
  print(download_path)


# configure LLM
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
text_gen = pipeline("text-generation", model=model,device = -1, tokenizer=tokenizer, max_new_tokens=100)

/root/.cache/huggingface/hub/models--openai-community--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
/root/.cache/huggingface/hub/models--openai-community--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/flax_model.msgpack
/root/.cache/huggingface/hub/models--openai-community--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/merges.txt
/root/.cache/huggingface/hub/models--openai-community--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/model.safetensors
/root/.cache/huggingface/hub/models--openai-community--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/pytorch_model.bin
/root/.cache/huggingface/hub/models--openai-community--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/rust_model.ot
/root/.cache/huggingface/hub/models--openai-community--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tf_model.h5
/root/.cache/huggingface/hub/models--openai-community--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenize

# Function for generating response from LLM

In [95]:
# Function to generate a response from LLM

def generate_response(question, relevent_chuncks):
  context = "\n\n".join(relevent_chuncks)
  prompt = (
      "You are an assistant for question-answering tasks. Use the following pieces data/documents "
      "retrieved context to answer the question. If you don't know the answer, just say that you don't know"
      "Use three sentences maximum and keep the answer concise."
      "\n\nContext:\n" + context + "\n\nQuestion:\n" + question
  )
  response = text_gen(prompt)
  return response[0]['generated_text']


Front End System

In [ ]:
# Example query

question = "What is the capital of France?"
relevent_chunks = query_documents(question)
response = generate_response(question, relevent_chunks)
print(response)